In [1]:
import os 
import numpy as np
import pandas as pd
import importlib as imp

from tqdm import tqdm, tqdm_notebook
import warnings
warnings.simplefilter('ignore')
pd.options.display.max_columns = 100

- **p01_c.txt**, the knapsack capacity. <br>
- **p01_w.txt**, the weights of the objects. <br>
- **p01_p.txt**, the profits of each object. <br>
- **p01_s.txt**, the optimal selection of weights. <br>

In [90]:
from utils import tools
tools = imp.reload(tools)

In [3]:
%%time
n = np.arange(1,8)
benchmarks = {k: tools.get_knapsack(n='0'+str(k)) for k in n}

01 is loaded!
02 is loaded!
03 is loaded!
04 is loaded!
05 is loaded!
06 is loaded!
07 is loaded!
Wall time: 29.8 s


In [22]:
benchmarks[1]

{'capacity': [165],
 'weights': [23, 31, 29, 44, 53, 38, 63, 85, 89, 82],
 'profits': [92, 57, 49, 68, 60, 43, 67, 84, 87, 72],
 'optimal': [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]}

In [562]:
from utils import tools
tools = imp.reload(tools)
from algorithm import brute_force, greedy, genetic, bound_and_branches, dynamic
brute_force = imp.reload(brute_force)
greedy      = imp.reload(greedy)
genetic     = imp.reload(genetic)

In [563]:
i = 6
knap = benchmarks[i]

param = {'n_epoch': 5000,
         'eps': 0.5,
         'chaos': 20,
         'n_chrom': 15}
knap = dict(knap, **param)
alg = genetic.Genetic(knap)
opt = alg.solve()
print('{} optimal solution {}'.format(alg.name, opt))
alg_knap = tools.compute_knapsack(knap, opt)
real_knap = tools.compute_knapsack(knap, knap['optimal'])
print('{} optimal profit {} and weight {}'.format(alg.name, alg_knap[1], alg_knap[0]))
print('Real optimal profit {} and maximum weight {}'.format(real_knap[1],real_knap[0]))

Genetic optimal profit 1735 and weight 169
Real optimal profit 1735 and maximum weight 169


In [603]:
import pprint
for key in benchmarks:
    
    knapsack = benchmarks[key]
    z = 30
    opt_w, opt_p = tools.compute_knapsack(knapsack, knapsack['optimal'])
    print('--- \n')
    print('\n' + '## Knapsack 0{}'.format(key))
    print(' - capacity: *{}*<br>'.format(knapsack['capacity'][0]))
    print(' - optimal solution: ```{}```<br>'.format(knapsack['optimal']))
    print(' - optimal weight: *{}*, and profit: *{}*<br>'.format(opt_w, opt_p))

    param = {'n_epoch': 5000,
             'eps': 0.5,
             'chaos': 20,
             'n_chrom': 15}
    algorithms = [brute_force.BruteForce,
                  greedy.Greedy,
                  bound_and_branches.BranchAndBound,
                  dynamic.Dynamic,
                  genetic.Genetic]
    for alg in algorithms:
        knapsack = dict(knapsack, **param)
        alg     = alg(knapsack)
        alg_opt = alg.solve()
        alg_opt_w, alg_opt_p = tools.compute_knapsack(knapsack, alg_opt)
        print('   **{}** optimal solution: ```{}```<br>'.format(alg.name, alg_opt))
        print('          optimal weight: {}, and profit **{}**'.format(alg_opt_w, alg_opt_p))



--- 


## Knapsack 01
 - capacity: *165*<br>
 - optimal solution: ```[1, 1, 1, 1, 0, 1, 0, 0, 0, 0]```<br>
 - optimal weight: *165*, and profit: *309*<br>
   **BruteForce** optimal solution: ```[1, 1, 1, 1, 0, 1, 0, 0, 0, 0]```<br>
          optimal weight: 165, and profit **309**
   **Greedy** optimal solution: ```[1, 1, 1, 1, 0, 1, 0, 0, 0, 0]```<br>
          optimal weight: 165, and profit **309**
   **Branch-And-Bound** optimal solution: ```[1, 1, 1, 1, 0, 1, 0, 0, 0, 0]```<br>
          optimal weight: 165, and profit **309**
   **Dynamic** optimal solution: ```[1, 1, 1, 1, 0, 1, 0, 0, 0, 0]```<br>
          optimal weight: 165, and profit **309**
   **Genetic** optimal solution: ```[1, 1, 1, 1, 0, 1, 0, 0, 0, 0]```<br>
          optimal weight: 165, and profit **309**
--- 


## Knapsack 02
 - capacity: *26*<br>
 - optimal solution: ```[0, 1, 1, 1, 0]```<br>
 - optimal weight: *26*, and profit: *51*<br>
   **BruteForce** optimal solution: ```[0, 1, 1, 1, 0]```<br>
          optim

In [564]:
genetic_param = {'n_epoch': 500,
                 'eps': 0.3,
                 'chaos': 20,
                 'n_chrom': 15}

algorithms = [brute_force.BruteForce,
              greedy.Greedy,
              bound_and_branches.BranchAndBound,
              dynamic.Dynamic,
              genetic.Genetic]

alg_params = [{},
              {},
              {},
              {},
              genetic_param]


stat_df = tools.generate_stat(algorithms, 
                        benchmarks, 
                        alg_params, 
                        n_observations=5)

In [595]:
stat = stat_df.groupby(['benchmark', 'algorithm']).agg({'execution':['mean','std'],
                                                        'capacity': 'median',
                                                        'optim_weight': 'median',
                                                        'optim_profit': 'median'})
stat.columns = [ 'execution mean', 'execution std', 'capacity','optim_weight', 'optim_profit']
stat = np.around(stat, 4).reset_index()
stat = stat.set_index(['benchmark'])
print(stat.to_markdown())

|   benchmark | algorithm        |   execution mean |   execution std |   capacity |   optim_weight |   optim_profit |
|------------:|:-----------------|-----------------:|----------------:|-----------:|---------------:|---------------:|
|           1 | Branch-And-Bound |           0      |          0      |        165 |            165 |            309 |
|           1 | BruteForce       |           0.0016 |          0.0005 |        165 |            165 |            309 |
|           1 | Dynamic          |           0.006  |          0.0007 |        165 |            165 |            309 |
|           1 | Genetic          |           0.0145 |          0.0089 |        165 |            165 |            309 |
|           1 | Greedy           |           0      |          0      |        165 |            165 |            309 |
|           2 | Branch-And-Bound |           0      |          0      |         26 |             26 |             51 |
|           2 | BruteForce       |           0  